## Prepare data

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv('titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Clean the data
# These columns (PassengerId, Name, Ticket, Cabin) are not useful for predicting survival. They either contain unique identifiers or non-informative text data that do not contribute to the model's predictive power.
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
# The 'Age' column has missing values. Using the median to fill these gaps is a common practice because it is robust to outliers and provides a central tendency measure.
df['Age'] = np.floor(df['Age'])
df['Age'].fillna(df['Age'].median(), inplace=True)

# The 'Embarked' column has missing values. Filling them with the most frequent value.
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')
df

/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_16571/12253161.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_16571/12253161.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


## Version 3 (huo)

In [ ]:
# Import libraries
import unittest
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import plotly.express as px
from dash import Dash, dcc, html
import matplotlib.pyplot as plt
import io
import base64

# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Clean the data
# These columns (PassengerId, Name, Ticket, Cabin) are not useful for predicting survival. They either contain unique identifiers or non-informative text data that do not contribute to the model's predictive power.
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
# The 'Age' column has missing values. Using the median to fill these gaps is a common practice because it is robust to outliers and provides a central tendency measure.
df['Age'] = np.floor(df['Age'])
df['Age'].fillna(df['Age'].median(), inplace=True)
# The 'Embarked' column has missing values. Filling them with the most frequent value.
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)
# Features and target: To prepare the data for modeling, we need to separate the features (input variables) from the target (output variable).
X = df.drop(columns=['Survived'])
y = df['Survived']

# Feature Scaling
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Models
models = {
    "Logistic Regression": [LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000)],
    "Decision Tree": [DecisionTreeClassifier(random_state=42), DecisionTreeClassifier(random_state=42)],
    "KNN (k=3)": [KNeighborsClassifier(n_neighbors=3), KNeighborsClassifier(n_neighbors=3)],
}

# train the model
def train_model(model, data_train):
    """Trains the model on the training data."""
    for data in data_train:
        X_train, y_train = data[:2]  # Only use the first two values
        model.fit(X_train, y_train)
    return model

# cross-validation splits data
def cross_validation_data(X, y):
    """Generates train-test splits for cross-validation."""
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    data_train = []
    data_test = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        data_train.append((X_train, y_train))
        data_test.append((X_test, y_test))

    return data_train, data_test

# cross-validation evaluation
def cross_validate_evaluation(trained_model, data_test):
    """Perform cross-validation and return metrics and confusion matrix."""
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []} # Store metrics for each fold
    confusion_matrices = [] # Store confusion matrices for each fold

    for X_test, y_test in data_test:
        # predict the data
        y_pred = trained_model.predict(X_test)
        metrics['accuracy'].append(accuracy_score(y_test, y_pred))
        metrics['precision'].append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
        metrics['recall'].append(recall_score(y_test, y_pred, average='weighted', zero_division=0))
        metrics['f1'].append(f1_score(y_test, y_pred, average='weighted', zero_division=0))
        confusion_matrices.append(confusion_matrix(y_test, y_pred))

    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)
    avg_metrics = {key: np.mean(value) for key, value in metrics.items()}
    
    return { # return a dictionary containing the average metrics and the average confusion matrix.
        **avg_metrics, # Unpack metrics
        'confusion_matrix': avg_confusion_matrix
        # 'decision_tree': dt_image
    }

# bootstrap splits data
def bootstrap_632_data(X, y):
    """Generates train-test splits for the Bootstrap .632 method."""
    data = []
    for _ in range(100):  # Number of bootstrap iterations
        X_boot, y_boot = resample(X, y, replace=True, n_samples=len(X), random_state=42)
        
        oob_indices = np.setdiff1d(np.arange(len(X)), np.unique(X_boot, return_index=True)[1])
        X_oob, y_oob = X[oob_indices], y[oob_indices]
        data.append((X_boot, y_boot, X_oob, y_oob, oob_indices))

    return data

# bootstrap evaluation
def bootstrap_632_evaluation(trained_model, data):
    train_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    test_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    confusion_matrices = []

    for X_boot, y_boot, X_oob, y_oob, oob_indices in data:

        y_pred_train = trained_model.predict(X_boot) 

        # Collect train metrics
        train_metrics['accuracy'].append(accuracy_score(y_boot, y_pred_train))
        train_metrics['precision'].append(precision_score(y_boot, y_pred_train, average='weighted', zero_division=0))
        train_metrics['recall'].append(recall_score(y_boot, y_pred_train, average='weighted', zero_division=0))
        train_metrics['f1'].append(f1_score(y_boot, y_pred_train, average='weighted', zero_division=0))

        # Collect test (OOB) metrics if there are out-of-bag samples
        if len(oob_indices) > 0:
            y_pred_oob = trained_model.predict(X_oob)
            test_metrics['accuracy'].append(accuracy_score(y_oob, y_pred_oob))
            test_metrics['precision'].append(precision_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['recall'].append(recall_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['f1'].append(f1_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            confusion_matrices.append(confusion_matrix(y_oob, y_pred_oob))

    # Aggregate train and test metrics
    train_metrics_mean = {metric: np.mean(train_metrics[metric]) for metric in train_metrics}
    test_metrics_mean = {metric: np.mean(test_metrics[metric]) for metric in test_metrics}

    # Combine metrics using the .632 formula
    combined_metrics = {metric: 0.368 * train_metrics_mean[metric] + 0.632 * test_metrics_mean[metric] for metric in train_metrics}
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0) if confusion_matrices else None

    combined_metrics['confusion_matrix'] = avg_confusion_matrix
    return combined_metrics

# decision tree visualization for prediction
def decision_tree_visualization(trained_model, X):
    """
    """
    buf = io.BytesIO()
    plt.figure(figsize=(15, 10))  # Reduced figure size for better app performance
    plot_tree(trained_model, feature_names=X.columns, class_names=["Not Survived", "Survived"], filled=True)
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)  # Reset buffer position
    return base64.b64encode(buf.getvalue()).decode("utf8")

def visualize_decision_path(trained_model, feature_names, class_names, sample_index, X_test, y_test):
    """
    Visualize the decision path of a trained Decision Tree for a specific test sample.
    """

    # Extract the decision path for the test sample
    node_indicator = trained_model.decision_path(X_test)
    feature = trained_model.tree_.feature
    threshold = trained_model.tree_.threshold

    # Select the sample's path
    sample_path = node_indicator.indices[
        node_indicator.indptr[sample_index]:node_indicator.indptr[sample_index + 1]
    ]

    # Print decision path for the sample
    print(f"Decision path for sample {sample_index}:")
    for node_id in sample_path:
        if feature[node_id] != -2:  # Not a leaf node
            print(f"Node {node_id}: If {feature_names[feature[node_id]]} <= {threshold[node_id]}")
        else:
            print(f"Node {node_id}: Leaf node")

    # Visualize the decision tree with the decision path highlighted
    buf = io.BytesIO()
    plt.figure(figsize=(15, 10))
    plot_tree(trained_model, feature_names=feature_names, class_names=class_names, filled=True)
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode("utf8")


# Evaluate models
crossval_results = {}
bootstrap_results = {}
decision_tree_cv = None
decision_tree_bs = None

# Convert X_scaled to DataFrame for compatibility
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)


# train the model, evaluate the model and store the results in the iteration
for model_name, model_pair in models.items():
    # Cross-validation
    data_train_cv, data_test_cv = cross_validation_data(X_scaled, y)
    trained_model_cv = train_model(model_pair[0], data_train_cv)
    crossval_results[model_name] = cross_validate_evaluation(trained_model_cv, data_test_cv)
    
    # Bootstrap
    data_bs = bootstrap_632_data(X_scaled, y)
    trained_model_bs = train_model(model_pair[1], data_bs)
    bootstrap_results[model_name] = bootstrap_632_evaluation(trained_model_bs, data_bs)

    # Store decision tree visualization if the model is DecisionTreeClassifier
    if isinstance(model_pair[0], DecisionTreeClassifier):
        decision_tree_cv = visualize_decision_path(trained_model_cv, X.columns, ["Not Survived", "Survived"], 0, X_scaled, y)
        decision_tree_bs = visualize_decision_path(trained_model_bs, X.columns, ["Not Survived", "Survived"], 0, X_scaled, y)


# 3. Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Titanic Classification Models Evaluation", style={'textAlign': 'center'}),

    # Metrics Comparison
    html.H3("Cross-Validation Metrics Comparison"),
    dcc.Graph(figure=px.bar(
        pd.DataFrame([
            {'Metric': metric, 'Model': model, 'Score': crossval_results[model][metric]}
            for model in models.keys()
            for metric in ['accuracy', 'precision', 'recall', 'f1']
        ]),
        x='Metric', y='Score', color='Model', barmode='group',
        title="Cross-Validation Metrics Comparison",
        labels={'Metric': 'Metrics', 'Score': 'Score'}
    )),

    html.H3("Bootstrap Metrics Comparison"),
    dcc.Graph(figure=px.bar(
        pd.DataFrame([
            {'Metric': metric, 'Model': model, 'Score': bootstrap_results[model][metric]}
            for model in models.keys()
            for metric in ['accuracy', 'precision', 'recall', 'f1']
        ]),
        x='Metric', y='Score', color='Model', barmode='group',
        title="Bootstrap Metrics Comparison",
        labels={'Metric': 'Metrics', 'Score': 'Score'}
    )),

    # Confusion Matrices
    html.H3("Confusion Matrices: Cross-Validation"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(crossval_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Blues').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),
    
    html.H3("Confusion Matrices: Bootstrap"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(bootstrap_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Reds').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),

    # Decision Tree visualisierung
    html.H3("Decision Tree(Crossvalidation)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_cv), style={'width': '100%', 'height': 'auto'}),
    html.H3("Decision Tree(Bootstrap)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_bs), style={'width': '100%', 'height': 'auto'}),

])

if __name__ == '__main__':
    app.run_server(debug=True)

## Version4 (tina)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier, plot_tree
import plotly.express as px
from dash import Dash, dcc, html
import io
import base64
import matplotlib.pyplot as plt


# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Clean the data
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
df['Age'] = np.floor(df['Age'])
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)
X = df.drop(columns=['Survived'])
y = df['Survived']

# Feature Scaling
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Models array (2D array for Cross-Validation and Bootstrap)
models = {
    "Logistic Regression": [LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000)],
    "Decision Tree": [DecisionTreeClassifier(random_state=42), DecisionTreeClassifier(random_state=42)],
    "KNN (k=3)": [KNeighborsClassifier(n_neighbors=3), KNeighborsClassifier(n_neighbors=3)],
}

# Cross-validation method
def cross_validate_method(model, X, y):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    confusion_matrices = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx] if isinstance(X, pd.DataFrame) else (X[train_idx], X[test_idx])
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx] if isinstance(y, pd.Series) else (y[train_idx], y[test_idx])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics['accuracy'].append(accuracy_score(y[test_idx], y_pred))
        metrics['precision'].append(precision_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        metrics['recall'].append(recall_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        metrics['f1'].append(f1_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        confusion_matrices.append(confusion_matrix(y[test_idx], y_pred))

    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)
    avg_metrics = {key: np.mean(value) for key, value in metrics.items()}

    return { 
        **avg_metrics,
        'confusion_matrix': avg_confusion_matrix
    }

# Bootstrap method
def bootstrap_632_method(model, X, y):
    train_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    test_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    confusion_matrices = []

    for _ in range(100):  
        # Resample for bootstrap
        X_boot, y_boot = resample(X, y, replace=True, n_samples=len(X), random_state=42)

        # Berechne die Out-of-Bag (OOB)-Indizes
        oob_indices = np.setdiff1d(np.arange(len(X)), np.unique(X_boot, return_index=True)[1])

        # Verwende .iloc[] für pandas DataFrame/Series 
        X_oob = X.iloc[oob_indices] if isinstance(X, pd.DataFrame) else X[oob_indices]
        y_oob = y.iloc[oob_indices] if isinstance(y, pd.Series) else y[oob_indices]

        # Trainiere das Modell mit den Bootstrap-Daten
        model.fit(X_boot, y_boot)
        y_pred_train = model.predict(X_boot)

        train_metrics['accuracy'].append(accuracy_score(y_boot, y_pred_train))
        train_metrics['precision'].append(precision_score(y_boot, y_pred_train, average='weighted', zero_division=0))
        train_metrics['recall'].append(recall_score(y_boot, y_pred_train, average='weighted', zero_division=0))
        train_metrics['f1'].append(f1_score(y_boot, y_pred_train, average='weighted', zero_division=0))

        if len(oob_indices) > 0:
            y_pred_oob = model.predict(X_oob)
            test_metrics['accuracy'].append(accuracy_score(y_oob, y_pred_oob))
            test_metrics['precision'].append(precision_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['recall'].append(recall_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['f1'].append(f1_score(y_oob, y_pred_oob, average='weighted', zero_division=0))
            confusion_matrices.append(confusion_matrix(y_oob, y_pred_oob))
            
    train_metrics_mean = {metric: np.mean(train_metrics[metric]) for metric in train_metrics}
    test_metrics_mean = {metric: np.mean(test_metrics[metric]) for metric in test_metrics}

    combined_metrics = {metric: 0.368 * train_metrics_mean[metric] + 0.632 * test_metrics_mean[metric] for metric in train_metrics}
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0) if confusion_matrices else None

    combined_metrics['confusion_matrix'] = avg_confusion_matrix
    return combined_metrics

# Visualize Decision Tree (cross-validation and bootstrap)
def decision_tree_visualization(model, X, y):
    # Visualize the Decision Tree
    buf = io.BytesIO()
    plt.figure(figsize=(100, 90))
    plot_tree(model, feature_names=X.columns, class_names=["Not Survived", "Survived"], filled=True)
    plt.savefig(buf, format="png")
    plt.close()
    return base64.b64encode(buf.getbuffer()).decode("utf8")

# Evaluate models
crossval_results = {}
bootstrap_results = {}

for model_name, model_pair in models.items():
    crossval_results[model_name] = cross_validate_method(model_pair[0], X, y)
    bootstrap_results[model_name] = bootstrap_632_method(model_pair[1], X, y)


# For decision tree visualization (cross-validation and bootstrap)
decision_tree_cv = decision_tree_visualization(models["Decision Tree"][0], X, y)
decision_tree_bootstrap = decision_tree_visualization(models["Decision Tree"][1], X, y)

# 3. Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Titanic Classification Models Evaluation", style={'textAlign': 'center'}),

    # Metrics Comparison
    html.H3("Cross-Validation Metrics Comparison"),
    dcc.Graph(figure=go.Figure(
        data=[go.Bar(
            x=['accuracy', 'precision', 'recall', 'f1'],
            y=[crossval_results[model][metric] for metric in ['accuracy', 'precision', 'recall', 'f1']],
            name=model
        ) for model in models.keys()],
        layout=go.Layout(
            barmode='group',
            title="Cross-Validation Metrics Comparison",
            xaxis=dict(title="Metric"),
            yaxis=dict(title="Score")
        )
    )),
    
    html.H3("Bootstrap Metrics Comparison"),
    dcc.Graph(figure=go.Figure(
        data=[go.Bar(
            x=['accuracy', 'precision', 'recall', 'f1'],
            y=[bootstrap_results[model][metric] for metric in ['accuracy', 'precision', 'recall', 'f1']],
            name=model
        ) for model in models.keys()],
        layout=go.Layout(
            barmode='group',
            title="Bootstrap Metrics Comparison",
            xaxis=dict(title="Metric"),
            yaxis=dict(title="Score")
        )
    )),
    html.H3("Confusion Matrices: Cross-Validation"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(crossval_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Blues').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),

    html.H3("Confusion Matrices: Bootstrap"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(bootstrap_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Reds').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),

    

    # Decision Tree visualizations
    html.H3("Decision Tree (Cross-Validation)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_cv), style={'width': '100%', 'height': 'auto'}),

    html.H3("Decision Tree (Bootstrap)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_bootstrap), style={'width': '100%', 'height': 'auto'})
])

if __name__ == '__main__':
    app.run_server(debug=True)


## Version 5 (huo)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier, plot_tree
import plotly.express as px
from dash import Dash, dcc, html
import io
import base64
import matplotlib.pyplot as plt


# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Clean the data
# These columns (PassengerId, Name, Ticket, Cabin) are not useful for predicting survival. They either contain unique identifiers or non-informative text data that do not contribute to the model's predictive power.
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
# The 'Age' column has missing values. Using the median to fill these gaps is a common practice because it is robust to outliers and provides a central tendency measure.
df['Age'] = np.floor(df['Age'])
df['Age'].fillna(df['Age'].median(), inplace=True)

# The 'Embarked' column has missing values. Filling them with the most frequent value.
# df['Embarked'] = df['Embarked'].astype('category')
# df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# df['Sex'] = df['Sex'].astype('category')

df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)


# Features and target: To prepare the data for modeling, we need to separate the features (input variables) from the target (output variable).
X = df.drop(columns=['Survived'])
y = df['Survived']

# Feature Scaling
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Models array (2D array for Cross-Validation and Bootstrap)
models = {
    "Logistic Regression": [LogisticRegression(max_iter=1000), LogisticRegression(max_iter=1000)],
    "Decision Tree": [DecisionTreeClassifier(random_state=42), DecisionTreeClassifier(random_state=42)],
    "KNN (k=3)": [KNeighborsClassifier(n_neighbors=3), KNeighborsClassifier(n_neighbors=3)],
}

# Cross-validation method
def cross_validate_method(model, X, y):
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    confusion_matrices = []
    # all_predictions = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx] if isinstance(X, pd.DataFrame) else (X[train_idx], X[test_idx])
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx] if isinstance(y, pd.Series) else (y[train_idx], y[test_idx])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        # all_predictions.append((X_test, y_test, y_pred))  # Store predictions and test labels

        metrics['accuracy'].append(accuracy_score(y[test_idx], y_pred))
        metrics['precision'].append(precision_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        metrics['recall'].append(recall_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        metrics['f1'].append(f1_score(y[test_idx], y_pred, average='weighted', zero_division=0))
        confusion_matrices.append(confusion_matrix(y[test_idx], y_pred))

    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)
    avg_metrics = {key: np.mean(value) for key, value in metrics.items()}

    return { 
        **avg_metrics,
        'confusion_matrix': avg_confusion_matrix
    }

# Bootstrap method
def bootstrap_632_method(model, X, y):
    train_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    test_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    confusion_matrices = []
    # all_predictions = []

    for _ in range(100):  
        # Resample for bootstrap
        X_train, y_train = resample(X, y, replace=True, n_samples=len(X), random_state=42)

        # get incides for out-of-bag samples (for test)
        test_indices = np.setdiff1d(np.arange(len(X)), np.unique(X_train, return_index=True)[1])

        # Verwende .iloc[] für pandas DataFrame/Series 
        X_test = X.iloc[test_indices] if isinstance(X, pd.DataFrame) else X[test_indices]
        y_test = y.iloc[test_indices] if isinstance(y, pd.Series) else y[test_indices]

        # Trainiere das Modell mit den Bootstrap-Daten
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)

        train_metrics['accuracy'].append(accuracy_score(y_train, y_pred_train))
        train_metrics['precision'].append(precision_score(y_train, y_pred_train, average='weighted', zero_division=0))
        train_metrics['recall'].append(recall_score(y_train, y_pred_train, average='weighted', zero_division=0))
        train_metrics['f1'].append(f1_score(y_train, y_pred_train, average='weighted', zero_division=0))

        if len(test_indices) > 0:
            
            y_pred_oob = model.predict(X_test)
            # all_predictions.append((X_test, y_test, y_pred_oob))  # Store predictions and test labels

            test_metrics['accuracy'].append(accuracy_score(y_test, y_pred_oob))
            test_metrics['precision'].append(precision_score(y_test, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['recall'].append(recall_score(y_test, y_pred_oob, average='weighted', zero_division=0))
            test_metrics['f1'].append(f1_score(y_test, y_pred_oob, average='weighted', zero_division=0))
            confusion_matrices.append(confusion_matrix(y_test, y_pred_oob))
            
    train_metrics_mean = {metric: np.mean(train_metrics[metric]) for metric in train_metrics}
    test_metrics_mean = {metric: np.mean(test_metrics[metric]) for metric in test_metrics}

    combined_metrics = {metric: 0.368 * train_metrics_mean[metric] + 0.632 * test_metrics_mean[metric] for metric in train_metrics}
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0) if confusion_matrices else None

    combined_metrics['confusion_matrix'] = avg_confusion_matrix
    return combined_metrics

# Visualize Decision Tree (cross-validation and bootstrap)
def decision_tree_visualization(model, feature_names):
    # Visualize the Decision Tree
    buf = io.BytesIO()
    plt.figure(figsize=(100, 90))
    plot_tree(model, feature_names=feature_names)
    plt.savefig(buf, format="png")
    plt.close()
    return base64.b64encode(buf.getbuffer()).decode("utf8")

# Evaluate models
crossval_results = {}
bootstrap_results = {}

for model_name, model_pair in models.items():
    crossval_results[model_name] = cross_validate_method(model_pair[0], X, y)
    bootstrap_results[model_name] = bootstrap_632_method(model_pair[1], X, y)


# For decision tree visualization (cross-validation and bootstrap)
decision_tree_cv = decision_tree_visualization(models["Decision Tree"][0], X.columns)
decision_tree_bootstrap = decision_tree_visualization(models["Decision Tree"][1], X.columns)

# 3. Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Titanic Classification Models Evaluation", style={'textAlign': 'center'}),

    # Metrics Comparison
    html.H3("Cross-Validation Metrics Comparison"),
    dcc.Graph(figure=go.Figure(
        data=[go.Bar(
            x=['accuracy', 'precision', 'recall', 'f1'],
            y=[crossval_results[model][metric] for metric in ['accuracy', 'precision', 'recall', 'f1']],
            name=model
        ) for model in models.keys()],
        layout=go.Layout(
            barmode='group',
            title="Cross-Validation Metrics Comparison",
            xaxis=dict(title="Metric"),
            yaxis=dict(title="Score")
        )
    )),
    
    html.H3("Bootstrap Metrics Comparison"),
    dcc.Graph(figure=go.Figure(
        data=[go.Bar(
            x=['accuracy', 'precision', 'recall', 'f1'],
            y=[bootstrap_results[model][metric] for metric in ['accuracy', 'precision', 'recall', 'f1']],
            name=model
        ) for model in models.keys()],
        layout=go.Layout(
            barmode='group',
            title="Bootstrap Metrics Comparison",
            xaxis=dict(title="Metric"),
            yaxis=dict(title="Score")
        )
    )),
    html.H3("Confusion Matrices: Cross-Validation"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(crossval_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Blues').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),

    html.H3("Confusion Matrices: Bootstrap"),
    html.Div([
        html.Div([
            dcc.Graph(figure=px.imshow(bootstrap_results[model]['confusion_matrix'], text_auto=True,
                                        title=f"{model}",
                                        color_continuous_scale='Reds').update_layout(
                                            autosize=False,
                                            width=300,
                                            height=300
                                        ))
        ], style={'margin': '10px'})
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),

    

    # Decision Tree visualizations
    html.H3("Decision Tree (Cross-Validation)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_cv), style={'width': '100%', 'height': 'auto'}),

    html.H3("Decision Tree (Bootstrap)"),
    html.Img(src="data:image/png;base64,{}".format(decision_tree_bootstrap), style={'width': '100%', 'height': 'auto'})
])

if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_16571/2461985573.py:27: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



